In [ ]:
# Дан файл с некоторыми данными. Формат файла – произвольный. 
# Считайте данные из файла и запишите их в Mongo. 
# Реализуйте и выполните следующие запросы:
#   вывод первых 10 записей, отсортированных по убыванию по полю salary;
#   вывод первых 15 записей, отфильтрованных по предикату age < 30, отсортировать по убыванию по полю salary
#   вывод первых 10 записей, отфильтрованных по сложному предикату: 
#          (записи только из произвольного города, записи только из трех произвольно взятых профессий), 
#          отсортировать по возрастанию по полю age
#   вывод количества записей, получаемых в результате следующей фильтрации 
#          (age в произвольном диапазоне, year в [2019,2022], 50 000 < salary <= 75 000 || 125 000 < salary < 150 000). 
#   здесь и везде предполагаем вывод в JSON.

In [1]:
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from pymongo import MongoClient, collection
from bson import ObjectId


In [3]:
with open('task_1_item.text', 'r', encoding="utf-8") as file:
   content = file.read()
lines = content.splitlines()
for line in lines:
    print(line)   

job::Бухгалтер
salary::182221
id::7762013
city::Мадрид
year::2018
age::60
=====
job::Инженер
salary::71912
id::2201388
city::Валенсия
year::2006
age::46
=====
job::Водитель
salary::132339
id::1077957
city::Эльче
year::2003
age::26
=====
job::Программист
salary::108353
id::2410517
city::Тудела
year::2009
age::57
=====
job::Косметолог
salary::155702
id::5529258
city::Баку
year::2017
age::25
=====
job::Строитель
salary::103024
id::7211216
city::Тирана
year::2016
age::53
=====
job::Косметолог
salary::63837
id::2377272
city::Белград
year::2013
age::64
=====
job::Архитектор
salary::158429
id::3575685
city::Вальядолид
year::2007
age::63
=====
job::Учитель
salary::59245
id::2828286
city::Кишинев
year::2010
age::50
=====
job::Бухгалтер
salary::104115
id::7813226
city::Махадаонда
year::2005
age::58
=====
job::Косметолог
salary::121185
id::4565250
city::Алма-Ата
year::2015
age::25
=====
job::Оператор call-центра
salary::136141
id::8451366
city::Сьюдад-Реаль
year::2009
age::29
=====
job::Программи

In [5]:
def read_data_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    data = []
    current_item = {}
    for line in lines:
        line = line.strip()
        if line.startswith("====="):
            if current_item:
                data.append(current_item)
                current_item = {}
        else:
            parts = line.split("::")
            if len(parts) == 2:
                key, value = parts
                current_item[key] = value

    if current_item:
        data.append(current_item)


    return data
print ()

In [6]:
def write_data_to_mongodb(data, host, port, db_name, collection_name):
    client = MongoClient(host, port)
    db = client[db_name]
    collection = db[collection_name]
    collection.insert_many(data)
    client.close()

In [7]:
def get_top_10_records_sorted_by_salary(collection):
    result = collection.find().sort('salary', -1).limit(10)
    return list(result)

In [8]:
def get_top_15_records_filtered_by_age(collection):
    result = collection.find({'age': {'$lt': 30}}).sort('salary', -1).limit(15)
    return list(result)

In [9]:
def get_top_10_records_filtered_by_city_and_jobs(collection, city, jobs):
    result = collection.find({'city': city, 'job': {'$in': jobs}}).sort('age', 1).limit(10)
    return list(result)

In [10]:
def get_records_count_filtered_by_conditions(collection, age_range, year_range, salary_range):
    result = collection.count_documents({
        '$and': [
            {'age': {'$gt': age_range[0], '$lt': age_range[1]}},
            {'year': {'$gte': year_range[0], '$lte': year_range[1]}},
            {'$or': [
                {'salary': {'$gt': salary_range[0], '$lte': salary_range[1]}},
                {'salary': {'$gt': salary_range[2], '$lt': salary_range[3]}}
            ]}
        ]
    })
    return result

In [11]:
def default_json_encoder(obj):
    if isinstance(obj, ObjectId):
        return str(obj)
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

In [12]:
def write_result_to_json(result, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(result, file, indent=4, ensure_ascii=False, default=default_json_encoder)

In [13]:
# Параметры
file_path = 'task_1_item.text'
host = 'localhost'
port = 27017
db_name = 'mydatabase'
collection_name = 'mycollection'
city = 'Мадрид'
jobs = ['Бухгалтер', 'Инженер', 'Программист']
age_range = (30, 50)
year_range = (2019, 2022)
salary_range = (50000, 75000, 125000, 150000)

In [14]:
# Чтение данных из файла
data = read_data_from_file(file_path)

In [15]:
# Запись данных в MongoDB
write_data_to_mongodb(data, host, port, db_name, collection_name)

In [16]:
# Подключение к MongoDB и получение коллекции
client = MongoClient(host, port)
db = client[db_name]
collection = db[collection_name]

In [17]:
# Вывод первых 10 записей, отсортированных по убыванию по полю salary
result1 = get_top_10_records_sorted_by_salary(collection)
write_result_to_json(result1, 'result1_1.json')
print (result1)

[{'_id': ObjectId('65a72102ef7de121883fc232'), 'job': 'Психолог', 'salary': '99934', 'id': '2466789', 'city': 'Монсон', 'year': '2003', 'age': '56'}, {'_id': ObjectId('65a71e13ef7de121883fbc9a'), 'job': 'Психолог', 'salary': '99934', 'id': '2466789', 'city': 'Монсон', 'year': '2003', 'age': '56'}, {'_id': ObjectId('65a7963a699e1bc7fc5bc373'), 'job': 'Психолог', 'salary': '99934', 'id': '2466789', 'city': 'Монсон', 'year': '2003', 'age': '56'}, {'_id': ObjectId('65a71ef3ef7de121883fbf66'), 'job': 'Психолог', 'salary': '99934', 'id': '2466789', 'city': 'Монсон', 'year': '2003', 'age': '56'}, {'_id': ObjectId('65a72118ef7de121883fc4fe'), 'job': 'Психолог', 'salary': '99934', 'id': '2466789', 'city': 'Монсон', 'year': '2003', 'age': '56'}, {'_id': ObjectId('65a72118ef7de121883fc4c7'), 'job': 'Менеджер', 'salary': '99910', 'id': '9399458', 'city': 'Белград', 'year': '2015', 'age': '58'}, {'_id': ObjectId('65a7963a699e1bc7fc5bc33c'), 'job': 'Менеджер', 'salary': '99910', 'id': '9399458', 'ci

In [18]:
# Вывод первых 15 записей, отфильтрованных по предикату age < 30, отсортировать по убыванию по полю salary
result2 = get_top_15_records_filtered_by_age(collection)
write_result_to_json(result2, 'result2_1.json')
print (result2)

[]


In [19]:
# Вывод первых 10 записей, отфильтрованных по сложному предикату
result3 = get_top_10_records_filtered_by_city_and_jobs(collection, city, jobs)
write_result_to_json(result3, 'result3_1.json')

In [20]:
# Вывод количества записей, получаемых в результате следующей фильтрации
result4 = get_records_count_filtered_by_conditions(collection, age_range, year_range, salary_range)
write_result_to_json(result4, 'result4_1.json')

In [105]:
# Закрытие соединения с MongoDB
client.close()